In [3]:
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import random
import pymorphy2

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Daria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
#список стран
countries = {
    'Франция': ['Европа', 'культура', 'отдых', 'горы', 'лыжи', 'море', 'водоёмы', 'сёрфинг'],
    'Испания' : ['Европа', 'культура', 'отдых', 'горы', 'лыжи', 'море', 'ночная жизнь'],
    'Италия' : ['Европа', 'культура', 'отдых', 'горы', 'лыжи', 'море', 'водоёмы'],
    'Великобритания': ['Европа', 'культура', 'море', 'водоёмы',  'сёрфинг'],
    'Турция' : ['Европа', 'культура', 'отдых', 'море'],
    'Греция' : ['Европа', 'культура', 'отдых', 'море', 'ночная жизнь'],
    'Нидерланды' : ['Европа', 'культура', 'море'],
    'Швейцария' : ['Европа', 'лыжи', 'горы', 'водоёмы'],
    'Австрия' : ['Европа', 'культура', 'отдых', 'горы', 'лыжи','водоёмы'],
    'Германия' : ['Европа', 'культура', 'горы', 'лыжи', 'ночная жизнь', 'водоёмы'],
    
    
    'США' : ['Америка', 'море', 'горы', 'пустыня', 'лыжи', 'отдых', 
             'культура', 'дикая природа', 'водоёмы',  'сёрфинг'],
    'Мексика' : ['Америка', 'море', 'горы','пустыня','древность', 'ночная жизнь', 'дайвинг'],
    
    
    'Китай' : ['Азия', 'море', 'горы','культура', 'ночная жизнь','пустыня',
               'дикая природа', 'водоёмы',  'сёрфинг'],
    'Таиланд' : ['Азия', 'море', 'отдых', 'ночная жизнь','водоёмы'],
    'Сингапур' : ['Азия', 'культура', 'ночная жизнь'],
    
    'Марокко' : ['Африка', 'море', 'горы',  'пустыня', 'ночная жизнь'],
    'ЮАР' : ['Африка', 'море', 'горы', 'пустыня', 'дикая природа', 'дайвинг',  'сёрфинг'],
    'Тунис' : ['Африка', 'море', 'горы', 'пустыня', 'отдых'],
    'Египет' : ['Африка', 'море', 'пустыня', 'культура', 'отдых', 'дайвинг'],
    'Зимбабве' : ['Африка', 'горы', 'дикая природа'],
    'Алжир' : ['Африка', 'море', 'горы', 'пустыня'],
    'ОАЭ' : ['Африка', 'море', 'отдых', 'культура', 'дикая природа']
}

clasterizator = {
    'дикая природа' : ['природа', 'джунгли', "лес"],
    'водоемы' : ['река', 'озеро'],
    'горы' : ['горы', 'водопад'],
    'пустыня' : ['пустыня'],
    'море' : ['море', 'океан'],
    'ночная жизнь' : ['бар', 'клуб', 'дискотека', 'тусовка', 'концерт'],
    'культура' : ['музей', 'театр', 'опера', 'балет', 'архитектура', 'церковь'],
    'отдых' : ['пляж', 'бассейн', 'загорать', 'отель'],
    'лыжи' : ['лыжи', 'горнолыжный курорт', 'сноуборд'],
    'дайвинг' : ['дайвинг'],
    'серфинг' : ['серфинг'] 
}

dialog_tree = {
    'природа' : ['пустыня', 'горы', 'дикая природа', 'водоем', 'море'],
    'расслабленный отдых' : ['отдых'],
    'город' : ['ночная жизнь', 'культура'],
    'активный отдых' : ['дайвинг', 'серфинг', 'лыжи']
}

continents = ['Америка', 'Африка', 'Европа', 'Азия' ]

positive_keys = ['да', 'нравится', 'конечно', 'ага', 'супер', 'круто', 'очень', 'отлично']
negative_keys = ['нет', 'не', 'фу', 'неа', 'ни', 'отстой']

questions = [
    ['Вы любите', 'accs'],
    ['Вам нравится', 'nomn'],
    ['Нравится ли Вам', 'nomn'],
    ['Любите ли Вы', 'accs'],
    ['Заинтересованы ли Вы в', 'loct'],
    ['Вас интересует', 'nomn']
]

answers = [
    ['Идеальным вариантом для Вас будет ', 'nomn'],
    ['Вам подойдет ', 'nomn'],
    ['Вам стоит поехать в ', 'accs']
]

stage = 0
relust_continents = []
result_keys = []
# 0 - не рассматривали, 1 - в процессе рассмотрения, 2 - отвергнута, 3 - принята
group_color = {}
group_score = {}
group_used = {}
key_color = {}
for group in list(dialog_tree.keys()):
    group_color[group] = 0
    group_score[group] = 0
    group_used[group] = 0
for key in list(clasterizator.keys()):
    key_color[key] = 0
current_group = ''
current_key = ''

In [12]:
def reverse_map(m):
    d = {}
    for key, value in m.items():
        for item in value:
            d[item] = key
    return d

reversed_clasterizator = reverse_map(clasterizator)
reversed_dialog_tree = reverse_map(dialog_tree)

def proccess_text(text):
    tokens = nltk.word_tokenize(text)
    morph = pymorphy2.MorphAnalyzer()
    words = []
    for word in tokens:
        words.append(morph.parse(word)[0].normal_form)
    return words

def get_key_words(words):
    key_words = []
    for word in words:
        if word in reversed_clasterizator:
            key_words.append(reversed_clasterizator[word])
    return key_words

def get_groups(words):
    groups = []
    for word in words:
        if word in dialog_tree and word != 'отдых':
            groups.append(word)
    return groups

In [13]:
def response(text, stage):
    
    user_text = proccess_text(text)
    morph = pymorphy2.MorphAnalyzer()
    
    # Приветствие
    if stage == 0:
        #print('---0')
        return 1, 'Здравствуйте! Я помогу Вам выбрать куда отправиться в путешествие. Готовы начать?'
    # Вопрос про континент
    if stage == 1:
        #print('---1')
        return 2, 'На какой бы континент Вы бы хотели поехать?'
    # Обработка ответа про континент
    if stage == 2:
        #print('---2')
        for word in user_text:
            if word in continents:
                result_continents.append(word)
        return response(text, stage = 3)
    # Вопрос про группу
    if stage == 3:
        #print('---3')
        if len(result_keys) >= 3:
            return -1, ''
        
        group = random.choice(list(dialog_tree.keys()))
        while group_color[group] != 0:
            group = random.choice(list(dialog_tree.keys()))
        global current_group
        current_group = group
        #global group_color
        group_color[group] = 1
        qst = random.choice(questions)
        return 4, qst[0] + ' ' + morph.parse(group)[0].inflect({qst[1]})[0] + '?'
    # Обработка ответа про группу
    if stage == 4:
        #print('---4')
        for key in negative_keys:
            if key in user_text:
                #global group_color
                group_color[current_group] = 2
                return response(text, stage = 3)
        key_words = get_key_words(user_text)
        has_keys = 0
        not_finished = ''
        for word in key_words:
            has_keys = 1
            global key_color 
            key_color[word] = 3
            result_keys.append(word)
            group = reversed_dialog_tree[word]
            global group_score
            group_score[group] += 1
            global group_used 
            group_used[group] += 1
            if group_score[group] >= 2 or group_used[group] == len(dialog_tree[group]):
                #global group_color
                group_color[group] = 3
            else:
                not_finished = group
        if has_keys:
            if not_finished == '':
                return response(text, stage = 3)
            else:
                #global current_group
                current_group = not_finished
                return response(text, stage = 5)
        for key in positive_keys:
            if key in user_text:
                return response(text, stage = 5)
    # Вопрос про ключ
    if stage == 5:
        #print('---5')
        if len(result_keys) >= 3:
            return -1, ''
        
        if group_used[current_group] == len(dialog_tree[current_group]):
            if group_score[current_group] > 0:
                #global group_color
                group_color[current_group] = 3
            else:
                #global group_color
                group_color[current_group] = 2
            return response(text, stage = 3)
        key = random.choice(dialog_tree[current_group])
        while key_color[key] != 0:
            key = random.choice(dialog_tree[current_group])
        word = random.choice(list(clasterizator[key]))
        global current_key
        current_key = key
        #global key_color
        key_color[key] = 1
        qst = random.choice(questions)
        return 6, qst[0] + ' ' + morph.parse(word)[0].inflect({qst[1]})[0] + '?'
    # Обработка ответа про ключ
    if stage == 6:
        #print('---6')
        for key in negative_keys:
            if key in user_text:
                #global key_color
                key_color[current_key] = 2
                group_used[reversed_dialog_tree[current_key]] += 1
                return response(text, stage = 5)
        key_words = get_key_words(user_text)
        has_keys = 0
        not_finished = ''
        for word in key_words:
            has_keys = 1
            #global key_color
            key_color[word] = 3
            result_keys.append(word)
            group = reversed_dialog_tree[word]
            #global group_score
            group_score[group] += 1
            #global group_used
            group_used[group] += 1
            if group_score[group] >= 2 or group_used[group] == len(dialog_tree[group]):
                #global group_color
                group_color[group] = 3
            else:
                not_finished = group                    
        if has_keys:
            if not_finished == '':
                return response(text, stage = 3)
            else:
                #global current_group
                current_group = not_finished
                return response(text, stage = 5)
        for key in positive_keys:
            if key in user_text:
                key_color[current_key] = 3
                result_keys.append(current_key)
                group = reversed_dialog_tree[current_key]
                group_score[group] += 1
                group_used[group] += 1
                if group_score[group] >= 2 or group_used[group] == len(dialog_tree[group]):
                    return response(text, stage = 3)
                else:
                    return response(text, stage = 5)                        

In [14]:
def lemmatization(text):
    tokens = nltk.word_tokenize(text)
    morph = pymorphy2.MorphAnalyzer()
    for word in tokens:
        morph.parse(word)[0].normal_form
    return tokens

def filter_countries(text):
    tokens = nltk.word_tokenize(text)
    tokens = lemmatization(tokens)

def create_response(stage, event, text):
    if stage == -1:
        country = county_mining(relust_continents, result_keys)
        answ = random.choice(answers)
        text = answ[0] + morph.parse(country)[0].inflect({answ[1]})[0]
    tokens = lemmatization(text)
    #buttons = []
    #positive = ['да', 'нравится', 'конечно', 'ага']
    #negative = ['нет', 'не нравится', 'фу', 'не-а']
    #if '?' in tokens:
    #    idx1 = random.randint(0, len(positive)-1)
    #    idx2 = random.randint(0, len(negative)-1)
    #    if stage == 1:
    #        if(idx1 == 1):
    #            idx1 += 1
    #        buttons.append({'title' : positive[idx1]})
    #    elif stage == 2:
    #        for c in continents:
    #            buttons.append({'title' : c})
    #    else:
    #        buttons.append({'title' : positive[idx1]})
    #        buttons.append({'title' : negative[idx2]})
    return {
        'version': event['version'],
        'session': event['session'],
        'response': {
            'text': text,
            #'buttons' : buttons,
            'end_session': 'false'
        },
    }

In [15]:
def county_mining(continents, keywords):
    candidats = {}
    for continent in continents:
        for country, features in countries.items():
            if continent in features:
                candidats[country] = features
    candidats_score = {country : 0 for country in candidats}
    max_score = 0
    for key in keywords:
        for country, features in candidats.items():
            if key in features:
                candidats_score[country] += 1
                max_score = max(candidats_score[country], max_score)
    best_countries = []
    for country, score in candidats_score.items():
        if score == max_score:
            best_countries.append(country)
    idx = 0
    if len(best_countries) > 1:
        idx = random.randint(0, len(best_countries)-1)
    return best_countries[idx]

In [16]:
def handler(event, context):
    user_text = ''
    if 'request' in event and 'original_utterance' in event['request'] and len(event['request']['original_utterance']) > 0:
        user_text = event['request']['original_utterance']
    global stage
    stage, text = response(user_text, stage)
    return create_response(stage, event, text)